In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.utils import class_weight
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dropout, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving modis_flood_features_cleaned_v1.1.csv to modis_flood_features_cleaned_v1.1.csv


In [ ]:
df = pd.read_csv("modis_flood_features_cleaned_v1.1.csv")
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date')

In [ ]:
# Preprocessing
X = df.drop(columns=["date", "target", "flooded", "jrc_perm_water"])
y = df["target"]
X = X.apply(pd.to_numeric, errors='coerce').fillna(0)

# Train-test split stratified
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42)

# Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# SMOTE
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled, y_train)

# Reshape for GRU
X_train_3d = X_train_smote.reshape((X_train_smote.shape[0], 1, X_train_smote.shape[1]))
X_test_3d = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

In [ ]:
# Build GRU model
# Build baseline GRU (no Dense layer, simpler arch)
def build_gru_baseline(input_shape):
    model = Sequential()
    model.add(Input(shape=input_shape))
    model.add(GRU(32))
    model.add(Dropout(0.3))
    model.add(Dense(1, activation='sigmoid'))
    model.add(Dense(32, activation='relu'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['precision', 'f1_score','recall'])
    return model

# Hitung class weight
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train_smote),
    y=y_train_smote
)
class_weights = dict(enumerate(class_weights))

In [ ]:
model = build_gru_baseline((1, X_train_smote.shape[1]))
history = model.fit(
    X_train_3d, y_train_smote,
    epochs=10,
    batch_size=128,
    validation_split=0.2,
    callbacks=[EarlyStopping(patience=2, restore_best_weights=True)],
    class_weight=class_weights,
    verbose=1
)

Epoch 1/10
14225/14225 ━━━━━━━━━━━━━━━━━━━━ 55s 4ms/step - f1_score: 0.5457 - loss: 0.2790 - precision: 0.8246 - recall: 0.8824 - val_f1_score: 1.0000 - val_loss: 0.2694 - val_precision: 1.0000 - val_recall: 0.9190
Epoch 2/10
14225/14225 ━━━━━━━━━━━━━━━━━━━━ 82s 4ms/step - f1_score: 0.5455 - loss: 0.2219 - precision: 0.8572 - recall: 0.9192 - val_f1_score: 1.0000 - val_loss: 0.2212 - val_precision: 1.0000 - val_recall: 0.9407
Epoch 3/10
14225/14225 ━━━━━━━━━━━━━━━━━━━━ 58s 4ms/step - f1_score: 0.5471 - loss: 0.2085 - precision: 0.8649 - recall: 0.9286 - val_f1_score: 1.0000 - val_loss: 0.2216 - val_precision: 1.0000 - val_recall: 0.9385
Epoch 4/10
14225/14225 ━━━━━━━━━━━━━━━━━━━━ 56s 4ms/step - f1_score: 0.5476 - loss: 0.2016 - precision: 0.8696 - recall: 0.9318 - val_f1_score: 1.0000 - val_loss: 0.2112 - val_precision: 1.0000 - val_recall: 0.9428
Epoch 5/10
14225/14225 ━━━━━━━━━━━━━━━━━━━━ 80s 4ms/step - f1_score: 0.5472 - loss: 0.1970 - precision: 0.8724 - recall: 0.9342 - val_f1_sco

In [ ]:
# Evaluate on test set (threshold default = 0.5)
y_pred = model.predict(X_test_3d)
y_pred_label = (y_pred > 0.5).astype(int)
print(confusion_matrix(y_test, y_pred_label))
print(classification_report(y_test, y_pred_label, digits=4))

9201/9201 ━━━━━━━━━━━━━━━━━━━━ 19s 2ms/step
[[261318  23170]
 [   632   9312]]
              precision    recall  f1-score   support

           0     0.9976    0.9186    0.9564    284488
           1     0.2867    0.9364    0.4390      9944

    accuracy                         0.9192    294432
   macro avg     0.6421    0.9275    0.6977    294432
weighted avg     0.9736    0.9192    0.9390    294432

